In [2]:
import torch
from huggingface_hub import login
from transformers import AutoTokenizer

from llama_index.llms import HuggingFaceLLM
from llama_index import SQLDatabase,ServiceContext
from sqlalchemy import create_engine, Table, inspect, MetaData
from llama_index.indices.struct_store.sql_query import NLSQLTableQueryEngine
from IPython.display import Markdown, display

login("")

hf_model="mistralai/Mistral-7B-Instruct-v0.2"

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
tokenizer = AutoTokenizer.from_pretrained(hf_model, pad=True, max_length=4096, truncation=False)

locally_run = HuggingFaceLLM(
    model_name=hf_model,
    device_map="auto",
    max_new_tokens=1024,
    tokenizer=tokenizer,
    model_kwargs={
        "load_in_4bit": True,
        "max_length": 4096,
        "max_memory": {0: "6GiB", "cpu": "30GiB"},
        "use_safetensors": True,
        "pad_token_id": 2,
    }
)

MAX_INPUT_TOKEN_LENGTH = 512

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
from llama_index import set_global_service_context
engine = create_engine("mysql+pymysql://sa:.@192.168.1.100:3306/sakila?charset=utf8mb4")

service_context = ServiceContext.from_defaults(llm=locally_run,embed_model="local")
set_global_service_context(service_context)
sql_database = SQLDatabase(engine)

In [5]:
query_engine = NLSQLTableQueryEngine(sql_database, service_context=service_context)

In [6]:
response = query_engine.query("Which actors are in the film Autumn Crow?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1295: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')


In [7]:
response_template = """
## Answer
```
{response}
```
## Generated SQL Query
```
{sql}
```
"""

display(Markdown(response_template.format(
        response=str(response),
        sql=response.metadata["sql_query"],
    )))


## Answer
```
The actors in the film "Autumn Crow" are Dustin Tautou, Angela Hudson, and James Pitt.
```
## Generated SQL Query
```
SELECT a.first_name, a.last_name
FROM actor AS a
JOIN film_actor AS fa ON a.actor_id = fa.actor_id
JOIN film AS f ON fa.film_id = f.film_id
WHERE f.title = 'Autumn Crow';
```


"first_name","last_name"
"DUSTIN","TAUTOU"
"ANGELA","HUDSON"
"JAMES","PITT"

In [8]:
from llama_index.indices.struct_store import SQLTableRetrieverQueryEngine
from llama_index.objects import (
    SQLTableNodeMapping,
    ObjectIndex,
    SQLTableSchema,
)
from llama_index import VectorStoreIndex

inspector = inspect(engine)
table_names = inspector.get_table_names()

table_node_mapping = SQLTableNodeMapping(sql_database)

table_schema_objs = []
for table_name in table_names:
    table_schema_objs.append(SQLTableSchema(table_name=table_name))

obj_index = ObjectIndex.from_objects(
    table_schema_objs,
    table_node_mapping,
    VectorStoreIndex
)

In [9]:
query_engine = SQLTableRetrieverQueryEngine(
    sql_database,
    obj_index.as_retriever(similarity_top_k=1),
)

In [10]:
response = query_engine.query("What were the top 5 films rented?")

response.metadata

{'9d94521f-a0ed-4e70-9f43-b9c8353e9ffb': {},
 'sql_query': "SELECT AVG(rental_duration) as avg_rental_duration\nFROM film\nJOIN rental ON film.film_id = rental.film_id\nWHERE language.language = 'English'\nGROUP BY film.film_id;"}

In [11]:
display(Markdown(response_template.format(
        response=str(response),
        sql=response.metadata["sql_query"],
    )))


## Answer
```

I'm sorry, but the provided SQL query is not valid. The query is trying to calculate the average rental duration for English films, but it's missing the necessary information to determine the top 5 films rented. To answer the original question, we would need to modify the query to include the film title or name, and sort the results by the number of rentals or total rental duration, then limit the results to the top 5. Here's an example of how the query could be modified:
```
SELECT f.title, AVG(r.rental_duration) as avg_rental_duration
FROM film f
JOIN rental r ON f.film_id = r.film_id
JOIN language l ON f.film_id = l.film_id
WHERE l.language = 'English'
GROUP BY f.film_id, f.title
ORDER BY COUNT(*) DESC
LIMIT 5;
```
This query will return the title of the top 5 English films and their average rental duration.
```
## Generated SQL Query
```
SELECT AVG(rental_duration) as avg_rental_duration
FROM film
JOIN rental ON film.film_id = rental.film_id
WHERE language.language = 'English'
GROUP BY film.film_id;
```


In [12]:
del locally_run
torch.cuda.empty_cache()